In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
% matplotlib inline

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale


np.random.seed(42)

In [ ]:
boston = load_boston()
X, y = boston.data, boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

`np.cov()` Estimate a covariance matrix, given data and weights.

Covariance indicates the level to which two variables vary together. If we examine N-dimensional samples, X = [x_1, x_2, ... x_N]^T, then the covariance matrix element C_{ij} is the covariance of x_i and x_j. The element C_{ii} is the variance of x_i.


you can also use `pd.DataFrame.corr()` to compute pairwise correlation of columns.

In probability theory and statistics, the mathematical concepts of `covariance` and `correlation` are very similar. Both describe the degree to which two random variables or sets of random variables tend to deviate from their expected values in similar ways.

> Covariance is when two variables vary with each other, whereas Correlation is when the change in one variable results in the change in another variable.

In [ ]:
X_train_scaled = scale(X_train)
cov = np.cov(X_train_scaled, rowvar=False)

In [ ]:
plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(cov)
plt.xticks(range(X.shape[1]), boston.feature_names)
plt.yticks(range(X.shape[1]), boston.feature_names);

useful snippet to sort values of your heatmap

In [ ]:
from scipy.cluster import hierarchy
order = np.array(hierarchy.dendrogram(hierarchy.ward(cov), no_plot=True)['ivl'], dtype="int")

In [ ]:
plt.figure(figsize=(8, 8), dpi=100)
plt.imshow(cov[order, :][:, order])
plt.xticks(range(X.shape[1]), boston.feature_names[order])
plt.yticks(range(X.shape[1]), boston.feature_names[order]);

# Supervised feature selection

`f_regression` is a univariate linear regression tests for testing individual effect of each of many regressors. 

This is done in 2 steps:
1. The correlation between each regressor and the target is computed
2. It is converted to an F score then to a p-value.


The F-test of overall significance indicates whether your linear regression model provides a better fit to the data than a model that contains no independent variables.

An F-test is a type of statistical test that is very flexible. You can use them in a wide variety of settings.


----
A p-value is a measure of the probability that an observed difference could have occurred just by random chance. The lower the p-value, the greater the statistical significance of the observed difference.


If the p-value is less than the significance level, your sample data provide sufficient evidence to conclude that your regression model fits the data better than the model with no independent variables.

In [ ]:
from sklearn.feature_selection import f_regression
f_values, p_values = f_regression(X, y)

In [ ]:
fig, ax = plt.subplots(2, 1)
ax[0].set_title("F values")
ax[0].plot(f_values, 'o')
ax[1].set_title("p values")
ax[1].plot(p_values, 'o')
ax[1].set_yscale("log")

ax[1].set_xticks(range(X.shape[1]))
ax[1].set_xticklabels(boston.feature_names, rotation=50);
fig.tight_layout()

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFpr
from sklearn.linear_model import RidgeCV

select = SelectKBest(k=2, score_func=f_regression)
select.fit(X_train, y_train)
print(X_train.shape)
print(select.transform(X_train).shape)

# put it all in a pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score


all_features = make_pipeline(StandardScaler(), RidgeCV())
select_f = make_pipeline(StandardScaler(), 
                       SelectKBest(k=2, score_func=f_regression), 
                       RidgeCV())

In [ ]:
print('accuracy with feature selection - F test:')
np.mean(cross_val_score(select_f, X_train, y_train, cv=10))

In [ ]:
print('accuracy without feature selection:')
np.mean(cross_val_score(all_features, X_train, y_train, cv=10))

There are many more approaches for feature selection like `VarianceThreshold`, `mutual_info_regression`, and so on...

#### Beware not to use a regression scoring function with a classification problem, you will get useless results.


Let's try `mutual information` and compare it with `F-Test`.

**mutual information** is one of many quantities that measures how much one random variables tells us about another. It is a dimensionless quantity with (generally) units of bits, and can be thought of as the reduction in uncertainty about one random variable given knowledge of another.

F-test captures only linear dependency. On the other hand, mutual information can capture any kind of dependency between variables.

In [ ]:
from sklearn.feature_selection import mutual_info_regression

select_mi = make_pipeline(StandardScaler(), 
                       SelectKBest(k=2, score_func=mutual_info_regression), 
                       RidgeCV())

In [ ]:
print('accuracy with feature selection - mutual information:')
np.mean(cross_val_score(select_mi, X_train, y_train, cv=10))

In [ ]:
# compare MI score and F values

scores = mutual_info_regression(X_train, y_train)

fig = plt.figure(figsize=(8, 2))
line_f, = plt.plot(f_values, 'o', c='r')
plt.ylabel("F value")
ax2 = plt.twinx()
line_s, = ax2.plot(scores, 'o', alpha=.7)
ax2.set_ylabel("MI score")
plt.xticks(range(X.shape[1]), boston.feature_names)
plt.legend([line_s, line_f], ["Mutual info scores", "F values"], loc=(0, 1))

# Recursive feature elimination

recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features.That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

In [ ]:
from sklearn.feature_selection import RFECV

regressor = RidgeCV()
rfecv = RFECV(estimator=regressor, step=1, cv=5,
              scoring='r2')

rfecv.fit(X, y)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

## Remeber Lasso regression?

In [ ]:
from sklearn.linear_model import LassoCV

X_train_scaled = scale(X_train)
lasso = LassoCV().fit(X_train_scaled, y_train)
print(lasso.coef_)

In [ ]:
fig = plt.figure(figsize=(8, 2))
line_f, = plt.plot(f_values, 'o', c='r')
plt.ylabel("F value")
ax2 = plt.twinx()
ax2.set_ylabel("lasso coefficients")
line_s, = ax2.plot(np.abs(lasso.coef_), 'o', alpha=.7)
plt.xticks(range(X.shape[1]), boston.feature_names)
plt.legend([line_s, line_f], ["Lasso coefficients abs", "F values"], loc=(0, 1))

# Select from model + Lasso

Coefficients in multiple linear models represent the relationship between the given feature, 
X
i
 and the target, 
y
, assuming that all the other features remain constant

In [ ]:
coefs = pd.DataFrame(
    lasso.coef_,
    columns=['Coefficients'], index=boston.feature_names
).sort_values(by='Coefficients')

In [ ]:
coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model, small regularization')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

Now we want to select the two features which are the most important. SelectFromModel() allows for setting the threshold. Only the features with the coef_ higher than the threshold will remain. Here, we want to set the threshold slightly above the third highest coef_ calculated by LassoCV() from our data.

In [ ]:
importance = np.abs(coefs['Coefficients'])
importance

In [ ]:
idx_third = importance.argsort()[-3]
threshold = importance[idx_third] + 0.01

idx_features = (-importance).argsort()[:2]
idx_features, threshold


In [ ]:
from sklearn.feature_selection import SelectFromModel


sfm = SelectFromModel(lasso, threshold=threshold)

sfm_pipe = make_pipeline(StandardScaler(), 
                                sfm,
                                RidgeCV()
                        )
sfm_pipe.fit(X_train, y_train)

In [ ]:
print('accuracy with feature selection - SelectFromModel:')
np.mean(cross_val_score(sfm_pipe, X_train, y_train, cv=10))

# Generate interaction features.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2, interaction_only=True)
poly.fit_transform(X).shape, X.shape

In [ ]:
pipe_rfe_poly = make_pipeline(StandardScaler(),
                              poly,
                              RFECV(estimator=regressor, 
                                    step=1,
                                    cv=5,
                                    scoring='r2')
                             )
np.mean(cross_val_score(pipe_rfe_poly, X_train, y_train, cv=5))